In [1]:
import warnings
import pandas as pd
import numpy as np
import pathlib
import os
from faker import Faker

In [2]:
jin = {"nuts":'sack',"balls":"2 of them"}

for key,val in jin.keys:
    print(key,val)

TypeError: cannot unpack non-iterable builtin_function_or_method object

In [3]:
fake = Faker()
n_patrons = 1000
print(f"Building {n_patrons} patron accounts")
values = tuple(
    zip([fake.name() for _ in range(n_patrons)],
        [fake.address() for _ in range(n_patrons)],
        [fake.unique.email() for _ in range(n_patrons)]))

## CHAT GPT SUGGESTED ##

# Function to simulate book selection
# Function to simulate book selection
def simulate_book_selection(persons, library):
    selections = []
    for person in persons:
        if np.random.rand() < person['probability_of_reading']:
            # Calculate weights based on hidden preferences
            weights = np.array([person['hidden_preferences'].get(genre, 0) for genre in library['Genre']])

            # Normalize weights to make them probabilities
            weights /= weights.sum()

            selected_book = np.random.choice(library['BookID'], p=weights)
            selections.append((person['person_id'], selected_book))
    return selections

# Generate a list of book genres
book_genres = ['Fiction', 'Mystery', 'Sci-Fi', 'Romance', 'Non-Fiction', 'Fantasy']

# Create a library of books with genres
num_books = 1000
library = pd.DataFrame({
    'BookID': np.arange(1, num_books + 1),
    'Genre': np.random.choice(book_genres, size=num_books)
})

# Create a list of persons with assigned genres and hidden preferences
num_persons = 100
persons = []
for person_id in range(1, num_persons + 1):
    assigned_genres = np.random.choice(book_genres, size=np.random.randint(1, len(book_genres) + 1), replace=False)

    # Generate hidden preferences for each person
    hidden_preferences = {genre: np.random.rand() for genre in book_genres}

    probability_of_reading = 0.01  # Very low probability of reading for each person
    persons.append({
        'person_id': person_id,
        'assigned_genres': assigned_genres,
        'hidden_preferences': hidden_preferences,
        'probability_of_reading': probability_of_reading
    })

# Simulate book selections for 100,000 iterations
num_iterations = 100000
selected_books = simulate_book_selection(persons, library)

# Display a sample of the selected books
print("Sample of selected books:")
print(selected_books[:10])


Building 1000 patron accounts
Sample of selected books:
[(96, 25)]


In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Tokenize the text and remove stop words
    tokens = [word.lower() for word in word_tokenize(text) if word.isalnum() and word.lower() not in stopwords.words('english')]
    return set(tokens)

def weighted_jaccard_similarity(book1, book2):
    # Define weights for each text field
    weights = {'title': 3, 'authors': 2, 'publisher': 1.5, 'description': 1, 'genres': 1}

    # Preprocess each text field
    preprocessed_fields1 = {field: preprocess_text(book1[field]) for field in weights.keys()}
    preprocessed_fields2 = {field: preprocess_text(book2[field]) for field in weights.keys()}

    # Calculate weighted Jaccard similarity for each text field
    similarities = [(field, weights[field] * len(preprocessed_fields1[field].intersection(preprocessed_fields2[field])) /
                     len(preprocessed_fields1[field].union(preprocessed_fields2[field])))
                    for field in weights.keys()]

    # Combine individual similarities using the assigned weights
    overall_similarity = sum(similarity for _, similarity in similarities)

    return overall_similarity

# Example usage:
book1 = {
    'title': 'The Catcher in the Rye',
    'authors': 'J.D. Salinger',
    'publisher': 'Little, Brown and Company',
    'publish_year': 1951,
    'description': 'A classic novel about a teenage boy',
    'genres': 'Fiction, Coming-of-age'
}

book2 = {
    'title': 'To Kill a Mockingbird',
    'authors': 'Harper Lee',
    'publisher': 'J.B. Lippincott & Co.',
    'publish_year': 1960,
    'description': 'A novel set in the American South during the 1930s',
    'genres': 'Fiction, Southern Gothic'
}

similarity_score = weighted_jaccard_similarity(book1, book2)
print(f"Similarity Score: {similarity_score}")



Similarity Score: 0.4583333333333333


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Morri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Morri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsRegressor
import populate_books

books = populate_books.read_books_data()
tfidf = TfidfVectorizer(analyzer='word',
                      ngram_range=(1, 10),
                      stop_words = 'english')


Generating bookshelves...
Generated 363 bookshelves containing 40000 books


In [10]:
vecs = tfidf.fit_transform(books["Title"].drop_duplicates())


In [11]:
vecs

<26737x341661 sparse matrix of type '<class 'numpy.float64'>'
	with 458900 stored elements in Compressed Sparse Row format>

In [12]:
similarity_matrix = cosine_similarity(vecs, vecs)

MemoryError: Unable to allocate 5.33 GiB for an array with shape (26737, 26737) and data type float64

In [13]:
from gensim.models import doc2vec
from gensim.parsing.preprocessing import preprocess_documents


titles = preprocess_documents(books["Title"].drop_duplicates(),)
model = doc2vec.Doc2Vec(books["Title"].drop_duplicates(),
                        vector_size=100,
                        window=5,
                        min_count=1,
                        workers=4,
                        dm=0) # PV-DBOW

AttributeError: 'str' object has no attribute 'words'